In [119]:
import numpy as np

In [120]:
class Upper_Hessenberg_Form:
    def __init__(self, matrix):
        self.matrix = matrix
        self.x_vector = [] # List down x vector each iteration. To be printed out later
        self.w_vector = [] # List down w vector each iteration. To be printed out later
        self.v_vector = [] # List down v vector each iteration. To be printed out later
        self.p_matrix = [] # List down projection matrix each iteration. To be printed out later
        self.H_hat_matrix = [] # List down Household Reflector hat each iteration. To be printed out later
        self.H_matrix = [] # List down Household Reflector matrices each iteration. To be printed out later
        self.similar_matrix_formula = []
        self.similar_matrix = []
        self.num_rows, self.num_columns = self.GET_MATRIX_ROWS_COLUMNS(matrix)
        
    def GET_MATRIX_ROWS_COLUMNS(self, matrix):
        # Number of rows and columns in the matrix
        num_rows, num_cols = matrix.shape
        return num_rows, num_cols 
    
    def check_if_Upper_Hessenberg_Form(self, matrix):
        # Checks if the matrix is in Upper Hessenberg Form
        num_rows, num_columns = matrix.shape

        for i in range(num_rows):
            for j in range(num_columns):
                if i - j > 1 and matrix[i, j] != 0:
                    return False

        return True

    
    def Household_Reflector(self):
        matrix = self.matrix
        
        for i in range(self.num_columns):
            
            # Get x_vector
            x_vector = matrix[i+1:, i]
            self.x_vector.append(x_vector)
            
            # Get w_vector
            w_vector = np.zeros(len(x_vector))
            w_vector[0] = np.linalg.norm(x_vector)
            
            if x_vector[0] < 0 and w_vector[0] < 0:
                w_vector[0] = -1*w_vector[0]
            elif x_vector[0] > 0 and w_vector[0] > 0:
                w_vector[0] = -1*w_vector[0]
                

            self.w_vector.append(w_vector)
            
            # Get v_vector
            v_vector = w_vector - x_vector
            self.v_vector.append(v_vector)
            
            # Get P_matrix
            p_matrix = np.outer(v_vector, v_vector) / np.dot(v_vector, v_vector)
            self.p_matrix.append(p_matrix)
            
            # Get H hat matrix
            H_hat_matrix = np.eye(p_matrix.shape[0]) - 2*p_matrix
            self.H_hat_matrix.append(H_hat_matrix)
            H_hat_num_columns, H_hat_num_rows = self.GET_MATRIX_ROWS_COLUMNS(H_hat_matrix)
            
            # Get H matrix
            H_matrix = np.eye(self.num_columns)
            
            # Insert H hat to H matrix
            row_start, row_end = i+1, i + 1 + H_hat_num_rows 
            col_start, col_end = i+1, i + 1 + H_hat_num_columns 
            
            H_matrix[row_start:row_end, col_start:col_end] = H_hat_matrix
            self.H_matrix.append(H_matrix)
            
            # Compute similar matrix
            if i == 0:
                # Store the formula used "H1*A*H1"
                formula = "H1*A*H1"
                self.similar_matrix_formula.append(formula)
                
                # Do computation H1*A*H1
                computation = np.round(H_matrix @ matrix @ H_matrix, 10)
                self.similar_matrix.append(computation)
            else:
                # Store the formula used for H_i+1*(formula)*H_i+1
                formula = "H" + str(i+1) + "(" + self.similar_matrix_formula[i-1] + ")*H" + str(i+1)
                self.similar_matrix_formula.append(formula)
                
                # Do computation H_i+1*(formula)*H_i+1
                computation = np.round(H_matrix @ self.similar_matrix[i-1] @ H_matrix, 10)
                self.similar_matrix.append(computation)
            
            # Updating matrix
            matrix = computation 
            
            # Check if matrix is already an upper hessenberg 
            validation = self.check_if_Upper_Hessenberg_Form(matrix)
            
            if validation == True:
                break
                
        # return nothing       
        return None
    def print_household_reflector_computations(self):
        
        print("________SOLUTION_________")
        print("")
        print("")
        
        for i in range(len(self.x_vector)):
            print("\033[1mIteration \033[0m" + str(i+1) + ":")
            print("\033[1mx vector: \033[0m" + str(self.x_vector[i]))
            print("\033[1mw vector: \033[0m" + str(self.w_vector[i]))
            print("\033[1mv vector: \033[0m" + str(self.v_vector[i]))
            print("")
            print("\033[1mP matrix: \033[0m")
            print("")
            print(self.p_matrix[i])
            print("")
            print("\033[1mH hat matrix: \033[0m")
            print("")
            print(self.H_hat_matrix[i])
            print("")
            print("\033[1mH matrix: \033[0m")
            print("")
            print(self.H_matrix[i])
            print("")
            print("\033[1mFormula: \033[0m" + str(self.similar_matrix_formula[i]))
            print("")
            print("\033[1mResulting matrix: \033[0m")
            print("")
            print(self.similar_matrix[i])
            print("")
            print("")
            
        return None
            
    def get_hessenberg_form_matrix(self):
        
        resulting_matrix = self.Household_Reflector()
        validation = self.check_if_Upper_Hessenberg_Form(self.similar_matrix[-1])
        
        if validation == True:
            solution = self.print_household_reflector_computations()
            similarity = self.check_similarity_matrix_Hessenberg()
            return resulting_matrix
        else:
            return "failure to find upper hessenberg form"
        
        return None


    def check_similarity_matrix_Hessenberg(self):
        print("")
        print("______SIMILARITY TEST______")

        eigenvalues_A = np.linalg.eigvals(self.matrix)
        eigenvalues_H = np.linalg.eigvals(self.similar_matrix[-1])
        
        print("")
        print("\033[1mEigenvalues of matrix A: \033[0m")
        print("")
        print(eigenvalues_A)
        print("")
        print("\033[1mEigenvalues of it's Upper Hessenberg form: \033[0m")
        print("")
        print(eigenvalues_H)
        print("")

        # Check if the characteristic polynomials are the same

        if np.allclose(eigenvalues_A, eigenvalues_H, atol=1e-6):
            print("\033[1mSimilarity Result: \033[0m" + "Matrix A and it's upper Hessenberg form H are similar.")
        else:
            print("\033[1mSimilarity Result: \033[0m" + "Matrix A and it's upper Hessenberg form H are not similar.")
        
        return None

In [121]:
A_matrix = np.random.rand(6,6)

Solution = Upper_Hessenberg_Form(A_matrix)
Solution.get_hessenberg_form_matrix()

________SOLUTION_________


Iteration 1:
x vector: [0.95750878 0.21147587 0.5352166  0.96060544 0.51120524]
w vector: [-1.55951772  0.          0.          0.          0.        ]
v vector: [-2.5170265  -0.21147587 -0.5352166  -0.96060544 -0.51120524]

P matrix: 

[[0.80698875 0.06780169 0.17159683 0.30798157 0.1638985 ]
 [0.06780169 0.00569657 0.01441725 0.02587604 0.01377045]
 [0.17159683 0.01441725 0.03648808 0.06548872 0.03485112]
 [0.30798157 0.02587604 0.06548872 0.117539   0.06255071]
 [0.1638985  0.01377045 0.03485112 0.06255071 0.0332876 ]]

H hat matrix: 

[[-0.61397749 -0.13560338 -0.34319367 -0.61596314 -0.327797  ]
 [-0.13560338  0.98860686 -0.02883449 -0.05175208 -0.02754089]
 [-0.34319367 -0.02883449  0.92702383 -0.13097745 -0.06970225]
 [-0.61596314 -0.05175208 -0.13097745  0.764922   -0.12510142]
 [-0.327797   -0.02754089 -0.06970225 -0.12510142  0.9334248 ]]

H matrix: 

[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.         -0.613977